In [ ]:
import tensorflow.compat.v1 as tf
import numpy as np

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
iris_path = "/content/drive/MyDrive/Colab Notebooks/data/Iris.csv"
house_path = "/content/drive/MyDrive/Colab-Notebooks/data/house-price/train.csv"

<h1>------- Introduction -------</h1>

<h3>intro 1: data flow</h3>

In [ ]:
a = tf.placeholder(tf.int32)
b = tf.placeholder(tf.int32)
add = tf.add(a, b)

with tf.Session() as sess:
    output = sess.run(add, feed_dict={a:5, b:-10.5})
    print(output)

-5


<h3>intro 2: logistic regression</h3>

In [ ]:
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

print(x_train)
print(y_train)

[[0 0]
 [0 1]
 [1 0]
 [1 1]]
[[0]
 [0]
 [0]
 [1]]


In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 2])
t = tf.placeholder(tf.float32, shape=[None, 1])

In [ ]:
W = tf.Variable(tf.zeros(shape=[2,1]))
b = tf.Variable(tf.zeros(shape=[1]))

In [ ]:
y = tf.sigmoid(tf.matmul(x, W) + b)
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [ ]:
#compare y_true (t) and y_pred(y)
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
#compute accuary
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(1000):
        sess.run(train_step, feed_dict={
            x: x_train,
            t: y_train
        })

        if (epoch % 100 == 0):
            acc_val = sess.run(accuracy, feed_dict={
                x: x_train,
                t: y_train
            })
            print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))
            print("W: {}    b: {}".format(sess.run(W), sess.run(b)))
            
    print("y_proba; {}".format(sess.run(y, feed_dict={x:x_train, t:y_train})))

epoch: 0, Accuracy: 0.750000
W: [[0.]
 [0.]]    b: [-0.1]
epoch: 100, Accuracy: 1.000000
W: [[1.7671356]
 [1.7671356]]    b: [-2.9394417]
epoch: 200, Accuracy: 1.000000
W: [[2.702048]
 [2.702048]]    b: [-4.282485]
epoch: 300, Accuracy: 1.000000
W: [[3.345771]
 [3.345771]]    b: [-5.2272086]
epoch: 400, Accuracy: 1.000000
W: [[3.8412285]
 [3.8412285]]    b: [-5.9599466]
epoch: 500, Accuracy: 1.000000
W: [[4.244355]
 [4.244355]]    b: [-6.558406]
epoch: 600, Accuracy: 1.000000
W: [[4.5839767]
 [4.5839767]]    b: [-7.0637155]
epoch: 700, Accuracy: 1.000000
W: [[4.8771544]
 [4.8771544]]    b: [-7.5005603]
epoch: 800, Accuracy: 1.000000
W: [[5.1348853]
 [5.1348853]]    b: [-7.8849826]
epoch: 900, Accuracy: 1.000000
W: [[5.36469]
 [5.36469]]    b: [-8.228008]
y_proba; [[1.9648671e-04]
 [4.9049795e-02]
 [4.9049795e-02]
 [9.3120372e-01]]


<h1>------- Sprint Tensorflow -------</h1>

<h2>[Problem 1] Looking back on the scratch</h2>

Step to implement deep learning
<ol>
    <li>preprocess input data X, y</li>
    <li>choose activations functions</li>
    <li>choose initializer, optimizer and initialize the neural network</li>
    <li>iterate each epoch, get mini batch</li>
    <li>do forward and backward, update W and b</li>
    <li>update CEE</li>
</ol>


<h2>[Problem 2] Consider the correspondence between scratch and TensorFlow</h2>

Load data set

In [ ]:
"""
Binary classification of Iris dataset using neural network implemented in TensorFlow
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#Load dataset
df = pd.read_csv(iris_path)
display(df.shape)

(150, 6)

Proprocessing X, y

In [ ]:
#Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
display(X.shape)
display(y.shape)

(100, 4)

(100,)

In [ ]:
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)

# Convert label to number
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]
display(y.shape)

(100, 1)

In [ ]:
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
class GetMiniBatch:
    """
    Iterator to get a mini-batch
    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

The main differences between Tensorflow implementation and my previous implementation of deep neural network are:
<ol>
<li>Tensorflow describe a strongly linking graph of calculation and objects via example_net(x).</li>
<li>No more implementation of forward and backward in Tensorflow, Tensorflow automatically detects and updates crucial variables.</li>
<li>Fewer lines of code.</li>
<li>No more thinking about the order of code execution, just descriptions.</li>
<li>No more worry about the shape of the array when calculating.</li>
<li>Plug and play.</li>
</ol>

In [ ]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output
#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 7.0241, val_loss : 67.6860, acc : 0.375
Epoch 1, loss : 3.4241, val_loss : 23.4026, acc : 0.312
Epoch 2, loss : 1.9387, val_loss : 11.6681, acc : 0.375
Epoch 3, loss : 2.0917, val_loss : 13.1400, acc : 0.312
Epoch 4, loss : 1.7685, val_loss : 17.7284, acc : 0.312
Epoch 5, loss : 1.6097, val_loss : 12.9607, acc : 0.312
Epoch 6, loss : 1.4402, val_loss : 10.0593, acc : 0.312
Epoch 7, loss : 1.3704, val_loss : 9.4797, acc : 0.312
Epoch 8, loss : 1.2536, val_loss : 9.8518, acc : 0.312
Epoch 9, loss : 1.1476, val_loss : 8.5670, acc : 0.375
Epoch 10, loss : 1.0930, val_loss : 8.0430, acc : 0.375
Epoch 11, loss : 1.0412, val_loss : 7.8791, acc : 0.375
Epoch 12, loss : 0.9804, val_loss : 7.1233, acc : 0.375
Epoch 13, loss : 0.9326, val_loss : 6.7908, acc : 0.375
Epoch 14, loss : 0.8792, val_loss : 6.2492, acc : 0.375
Epoch 15, loss : 0.8304, val_loss : 5.7681, acc : 0.375
Epoch 16, loss : 0.7835, val_loss : 5.2886, acc : 0.438
Epoch 17, loss : 0.7384, val_loss : 4.8037, acc : 0

<h2>[Problem 3] Create a model of Iris using all three types of objective variables</h2>

In [ ]:
"""
Trinary classification of Iris dataset using neural network implemented in TensorFlow
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#Load dataset
df = pd.read_csv(iris_path)
df["Species"].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

In [ ]:
#Condition extraction from data frame
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
display(y.shape)
display(X.shape)

(150,)

(150, 4)

In [ ]:
# Convert label to one hot
y = pd.get_dummies(y)

# NumPy 配列に変換
X = np.array(X)
y = np.array(y)

y = y.astype(np.int64)
display(y.shape)
display(y[:5, :])

(150, 3)

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [ ]:
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output
#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0, loss : 11.3028, val_loss : 102.2222, acc : 0.528
Epoch 1, loss : 8.7998, val_loss : 81.4813, acc : 0.583
Epoch 2, loss : 6.8027, val_loss : 59.8930, acc : 0.694
Epoch 3, loss : 4.7791, val_loss : 39.9069, acc : 0.694
Epoch 4, loss : 3.0486, val_loss : 25.9002, acc : 0.472
Epoch 5, loss : 1.6053, val_loss : 10.0990, acc : 0.639
Epoch 6, loss : 0.4247, val_loss : 2.8406, acc : 0.806
Epoch 7, loss : 0.0561, val_loss : 1.1800, acc : 0.861
Epoch 8, loss : 0.0306, val_loss : 1.0144, acc : 0.917
Epoch 9, loss : 0.0183, val_loss : 0.7311, acc : 0.944
Epoch 10, loss : 0.0175, val_loss : 0.8111, acc : 0.944
Epoch 11, loss : 0.0155, val_loss : 0.7300, acc : 0.944
Epoch 12, loss : 0.0162, val_loss : 0.8065, acc : 0.944
Epoch 13, loss : 0.0145, val_loss : 0.7146, acc : 0.944
Epoch 14, loss : 0.0157, va

<h2>[Problem 4] Creating a model of House Prices</h2>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#Load dataset
df = pd.read_csv(house_path)
df["SalePrice"].value_counts()

140000    20
135000    17
145000    14
155000    14
190000    13
          ..
84900      1
424870     1
415298     1
62383      1
34900      1
Name: SalePrice, Length: 663, dtype: int64

In [ ]:
#Condition extraction from data frame
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
display(y.shape)
display(X.shape)

(1460,)

(1460, 2)

In [ ]:
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)

y = y.astype(np.int64)
display(y.shape)
display(y[:5])

(1460,)

array([208500, 181500, 223500, 140000, 250000])

In [ ]:
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
#do standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

display(X_train[:5, :])

array([[-0.80043595, -1.318735  ],
       [ 0.87578699, -1.93738445],
       [-0.02107061,  1.22098379],
       [ 0.28049286, -1.18849301],
       [-0.16989415,  0.96049981]])

In [ ]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output
#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.keras.losses.mean_squared_error(Y, logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
#correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(logits - 0.5))
#Indicator value calculation
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        #total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, total_loss, val_loss))
    test_err = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print("test_mean_square_error : {:.3f}".format(test_err))

Epoch 0, loss : 3948120381.9443, val_loss : 37120552960.0000
Epoch 1, loss : 3942776108.4026, val_loss : 37057601536.0000
Epoch 2, loss : 3934423912.7024, val_loss : 36954574848.0000
Epoch 3, loss : 3920518896.1028, val_loss : 36782608384.0000
Epoch 4, loss : 3897984047.1435, val_loss : 36513767424.0000
Epoch 5, loss : 3864252709.8244, val_loss : 36124147712.0000
Epoch 6, loss : 3816531318.9550, val_loss : 35584675840.0000
Epoch 7, loss : 3752163932.0942, val_loss : 34875674624.0000
Epoch 8, loss : 3669608737.4390, val_loss : 33987112960.0000
Epoch 9, loss : 3568014139.7516, val_loss : 32912873472.0000
Epoch 10, loss : 3447110648.3255, val_loss : 31655790592.0000
Epoch 11, loss : 3307708958.6981, val_loss : 30229235712.0000
Epoch 12, loss : 3151456034.5353, val_loss : 28652154880.0000
Epoch 13, loss : 2980598980.2484, val_loss : 26949185536.0000
Epoch 14, loss : 2797911360.1370, val_loss : 25149718528.0000
Epoch 15, loss : 2606559169.5075, val_loss : 23286319104.0000
Epoch 16, loss : 2

The difference between classification and regression implementation of NN:
<ul>
<li>
    Classification:
    <ul>
    <li>Using softmax activation function to output value based on probability</li>
    <li>Using cross entropy loss function</li>
    </ul>
</li>

<li>
    Regression:
    <ul>
    <li>Using linear function to regress a final value</li>
    <li>Using mean square error loss function</li>
    </ul>
</li>
</ul>

<h2>[Problem 5] Creating a MNIST model</h2>

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
display(X_train.shape)
display(X_test.shape)

(60000, 784)

(10000, 784)

In [ ]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

In [ ]:
# Convert label to one hot
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

# NumPy 配列に変換
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)

display(y_train.shape)
display(y_test.shape)

display(y_train[:5, :])

(60000, 10)

(10000, 10)

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

<h3>Using FC only</h3>

In [ ]:
# Hyperparameter settings
learning_rate = 0.001
batch_size = 20
num_epochs = 10
n_hidden1 = 400
n_hidden2 = 200
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output
#Read network structure                              
logits = example_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 4.9512, val_loss : 34.6165, acc : 0.981
Epoch 1, loss : 1.0398, val_loss : 22.8223, acc : 0.986
Epoch 2, loss : 0.4929, val_loss : 18.4501, acc : 0.987
Epoch 3, loss : 0.2546, val_loss : 15.4491, acc : 0.988
Epoch 4, loss : 0.1417, val_loss : 14.8346, acc : 0.988
Epoch 5, loss : 0.0819, val_loss : 14.8641, acc : 0.989
Epoch 6, loss : 0.0492, val_loss : 12.5961, acc : 0.990
Epoch 7, loss : 0.0360, val_loss : 12.5920, acc : 0.991
Epoch 8, loss : 0.0265, val_loss : 12.4742, acc : 0.991
Epoch 9, loss : 0.0213, val_loss : 12.0821, acc : 0.992
test_acc : 0.992


<h3>Using CNN</h3>

In [ ]:
# Hyperparameter settings
learning_rate = 0.01
batch_size = 200
num_epochs = 30
n_hidden1 = 120
n_hidden2 = 84
n_input = X_train.shape[-1]
n_samples = X_train.shape[0]
n_classes = 10

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def conv_net(x):
    tf.random.set_random_seed(1)

    weights = {
        'w1': tf.Variable(tf.random_normal([5, 5, 1, 6])),
        'w2': tf.Variable(tf.random_normal([5, 5, 6, 16])),
    }

    biases = {
        'b1': tf.Variable(tf.random_normal([6])),
        'b2': tf.Variable(tf.random_normal([16])),
    }

    x = tf.reshape(x, [-1, 28, 28, 1]) #1@28x28

    c1 = tf.add(tf.nn.conv2d(x, weights['w1'], [1,1,1,1], padding="VALID"), biases['b1']) #6@24x24
    c1 = tf.nn.relu(c1) #6@24x24

    s2 = tf.nn.pool(c1, window_shape=[2,2], pooling_type="MAX", strides=[2,2], padding="VALID") #6@12x12

    c3 = tf.add(tf.nn.conv2d(s2, weights['w2'], [1,1,1,1], padding="VALID"), biases['b2']) #16@8x8
    c3 = tf.nn.relu(c3) #16@10x10

    s4 = tf.nn.pool(c3, window_shape=[2,2], pooling_type="MAX", strides=[2,2], padding="VALID") #16@4x4
    #flattening
    layer_output = tf.reshape(s4, [-1, 16*4*4])
    return layer_output

def fc_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([x.shape[-1], n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output

def le_net(x):
    return fc_net(conv_net(x))

#Read network structure                              
logits = le_net(X)
# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
#Initialization of variable
init = tf.global_variables_initializer()

#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 3.1401, val_loss : 107.8022, val_acc : 0.974
Epoch 1, loss : 0.3384, val_loss : 53.8764, val_acc : 0.981
Epoch 2, loss : 0.1725, val_loss : 37.3705, val_acc : 0.985
Epoch 3, loss : 0.1092, val_loss : 29.6584, val_acc : 0.986
Epoch 4, loss : 0.0751, val_loss : 24.8350, val_acc : 0.987
Epoch 5, loss : 0.0536, val_loss : 21.1964, val_acc : 0.989
Epoch 6, loss : 0.0409, val_loss : 19.9643, val_acc : 0.989
Epoch 7, loss : 0.0304, val_loss : 17.6325, val_acc : 0.990
Epoch 8, loss : 0.0248, val_loss : 15.9675, val_acc : 0.990
Epoch 9, loss : 0.0188, val_loss : 15.6557, val_acc : 0.990
Epoch 10, loss : 0.0160, val_loss : 14.8198, val_acc : 0.990
Epoch 11, loss : 0.0137, val_loss : 14.2430, val_acc : 0.990
Epoch 12, loss : 0.0105, val_loss : 13.6983, val_acc : 0.990
Epoch 13, loss : 0.0105, val_loss : 14.5615, val_acc : 0.990
Epoch 14, loss : 0.0093, val_loss : 13.4303, val_acc : 0.991
Epoch 15, loss : 0.0080, val_loss : 11.8737, val_acc : 0.991
Epoch 16, loss : 0.0069, val_loss